In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import seaborn as sns
import sqlite3
import glob
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

/Users/hari/.pyenv/versions/3.6.5/envs/covid/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [93]:
def get_data():
    conn = sqlite3.connect("covid.db")
    df_all = pd.read_sql("select * from daily",conn)
    conn.close()
    df_all=df_all.sort_values('Date')
    df_all=df_all[['Date','Country','Confirmed','Deaths','Recovered']].fillna(0)
    df_all['Infected'] = df_all['Confirmed']-(df_all['Recovered']-df_all['Deaths'])
    return df_all


def predict_poly_reg(country,column,date):
    df = get_data()
    df = df[df['Confirmed']>0]
    le_date = preprocessing.LabelEncoder()
    le_date.fit(df['Date'].values)
    df['Date']=le_date.transform(df['Date']).reshape(-1, 1)
    le_cntr = preprocessing.LabelEncoder()
    le_cntr.fit(df['Country'].values)
    df['Country']=le_cntr.transform(df['Country']).reshape(-1, 1)

    X=df[['Date','Country']].values
    y=df[column].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    poly_reg = PolynomialFeatures(degree=4)
    X_poly = poly_reg.fit_transform(X_train)
    pol_reg = LinearRegression()
    pol_reg.fit(X_poly, y_train)

    error = mean_squared_error(y, pol_reg.predict(poly_reg.fit_transform(X)), squared=False)
    print("Error {0}".format(error))
    X_user=np.array([le_date.transform([date]),le_cntr.transform([country])]).reshape(-1,2)
    print(pol_reg.predict(poly_reg.fit_transform(X_user)))

def apply_random_forest(country,column,date):
    df = get_data()
    df = df[df['Confirmed']>0]
    le_date = preprocessing.LabelEncoder()
    le_date.fit(df['Date'].values)
    df['Date']=le_date.transform(df['Date']).reshape(-1, 1)
    le_cntr = preprocessing.LabelEncoder()
    le_cntr.fit(df['Country'].values)
    df['Country']=le_cntr.transform(df['Country']).reshape(-1, 1)

    X=df[['Date','Country']].values
    y=df[column].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    reg = RandomForestRegressor(max_depth=None,random_state=0)
    reg.fit(X_train,y_train)

    error = mean_squared_error(y, reg.predict(X), squared=False)
    print(error)
    X_user=np.array([le_date.transform([date]),le_cntr.transform([country])]).reshape(-1,2)
    print(reg.predict(X_user))

In [99]:
predict_poly_reg('Singapore','Confirmed','2020-04-11')

Error 18528.329709882753
[9604.78416318]


In [103]:
apply_random_forest('UK','Deaths','2020-04-11')

231.19224499309144
[1169.54]
